<a href="https://colab.research.google.com/github/jordanhchong/Big_Data_Science_Feature_Selection/blob/main/BDS_Feature_Selection_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive');
directory = '/content/drive/My Drive/Colab Notebooks/Breast_Cancer_dataset.csv'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

!pip install category_encoders

import category_encoders as ce

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
breast_cancer_dataframe = pd.read_csv(directory)
breast_cancer_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     4024 non-null   int64 
 1   Race                    4024 non-null   object
 2   Marital Status          4024 non-null   object
 3   T Stage                 4024 non-null   object
 4   N Stage                 4024 non-null   object
 5   6th Stage               4024 non-null   object
 6   differentiate           4024 non-null   object
 7   Grade                   4024 non-null   object
 8   A Stage                 4024 non-null   object
 9   Tumor Size              4024 non-null   int64 
 10  Estrogen Status         4024 non-null   object
 11  Progesterone Status     4024 non-null   object
 12  Regional Node Examined  4024 non-null   int64 
 13  Reginol Node Positive   4024 non-null   int64 
 14  Survival Months         4024 non-null   int64 
 15  Stat

# Preprocessing

In [ ]:
#one-hot encoding to change race and marital status to numerical values
ce_OHE = ce.OneHotEncoder(cols=['Race','Marital Status'])

#creates bolean values for each of the categorical values of Race and Marital Status features
data1 = ce_OHE.fit_transform(breast_cancer_dataframe)
data1 = data1.rename(columns={'Race_1': 'White', 'Race_2': 'Black', 'Race_3' : 'Other'})
data1 = data1.rename(columns={'Marital Status_1': 'Married', 'Marital Status_2': 'Divorced', 'Marital Status_3' : 'Single', 'Marital Status_4' : 'Widowed', 'Marital Status_5' : 'Separated'})
data1.head()

,Age,White,Black,Other,Married,Divorced,Single,Widowed,Separated,T Stage,...,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,1,0,0,1,0,0,0,0,T1,...,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,1,0,0,1,0,0,0,0,T2,...,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,1,0,0,0,1,0,0,0,T3,...,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,1,0,0,1,0,0,0,0,T1,...,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,1,0,0,1,0,0,0,0,T2,...,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [ ]:
#Change categorical features to numerical values based on severity
#higher numbers indicate more severe stage
T_stage_dict = {'T1':1, 'T2':2,'T3':3, 'T4':4}
N_stage_dict = {'N1':1, 'N2':2,'N3':3, 'N4':4}
sixth_stage_dict = {'IA':1, 'IB':2,'IIA':3, 'IIB':4,'IIIA':5,'IIIB':6, 'IIIC':7}
differentiate_dict = {'Undifferentiated': 0,'Poorly differentiated':1, 'Moderately differentiated':2,'Well differentiated':3}
grade_dict = {'1':1, '2':2, '3':3, ' anaplastic; Grade IV': 4}
#create boolean values for categorical features with only two values
regional_dict = {'Regional':0, 'Distant':1}
positive_dict = {'Positive':1, 'Negative':0}
status_dict = {'Alive':1, 'Dead':0}

# Replace values in column X based on the dictionary
data1['T Stage '] = data1['T Stage '].replace(T_stage_dict)
data1['N Stage'] = data1['N Stage'].replace(N_stage_dict)
data1['6th Stage'] = data1['6th Stage'].replace(sixth_stage_dict)
data1['differentiate'] = data1['differentiate'].replace(differentiate_dict)
data1['Grade'] = data1['Grade'].replace(grade_dict)
data1['A Stage'] = data1['A Stage'].replace(regional_dict)
data1['Estrogen Status'] = data1['Estrogen Status'].replace(positive_dict)
data1['Progesterone Status'] = data1['Progesterone Status'].replace(positive_dict)
data1['Status'] = data1['Status'].replace(status_dict)

data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Age                     4024 non-null   int64
 1   White                   4024 non-null   int64
 2   Black                   4024 non-null   int64
 3   Other                   4024 non-null   int64
 4   Married                 4024 non-null   int64
 5   Divorced                4024 non-null   int64
 6   Single                  4024 non-null   int64
 7   Widowed                 4024 non-null   int64
 8   Separated               4024 non-null   int64
 9   T Stage                 4024 non-null   int64
 10  N Stage                 4024 non-null   int64
 11  6th Stage               4024 non-null   int64
 12  differentiate           4024 non-null   int64
 13  Grade                   4024 non-null   int64
 14  A Stage                 4024 non-null   int64
 15  Tumor Size           

In [ ]:
data1 = data1.rename(columns={'T Stage ': 'T Stage'})

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

#detect outliers using LOF
clf = LocalOutlierFactor()
clf.fit_predict(data1)
outliers = clf.negative_outlier_factor_[clf.negative_outlier_factor_ < -2]
print(outliers)
#no outliers detected

[]


In [ ]:
#minMax normalization
normalized_df=(data1-data1.min())/(data1.max()-data1.min())
normalized_df.head()

,Age,White,Black,Other,Married,Divorced,Single,Widowed,Separated,T Stage,...,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,0.974359,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,...,0.333333,0.666667,0.0,0.021583,1.0,1.0,0.383333,0.000000,0.556604,1.0
1,0.512821,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,...,0.666667,0.333333,0.0,0.244604,1.0,1.0,0.216667,0.088889,0.575472,1.0
2,0.717949,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.666667,...,0.666667,0.333333,0.0,0.446043,1.0,1.0,0.216667,0.133333,0.698113,1.0
3,0.717949,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,...,0.333333,0.666667,0.0,0.122302,1.0,1.0,0.016667,0.000000,0.783019,1.0
4,0.435897,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,...,0.333333,0.666667,0.0,0.287770,1.0,1.0,0.033333,0.000000,0.462264,1.0


# Feature Selection

In [ ]:
#Remove columns with greater that 0.7 covariance
# Calculate correlation matrix
correlation_matrix = normalized_df.corr()

# Set threshold for correlation coefficient
threshold = 0.7

# Find pairs of features with correlation greater than threshold
correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

print("Highly correlated features:", correlated_features)

# Drop highly correlated features
df_filtered = normalized_df.drop(columns=correlated_features)

df_filtered

Highly correlated features: {'6th Stage', 'Reginol Node Positive', 'Tumor Size', 'Grade'}


,Age,White,Black,Other,Married,Divorced,Single,Widowed,Separated,T Stage,N Stage,differentiate,A Stage,Estrogen Status,Progesterone Status,Regional Node Examined,Survival Months,Status
0,0.974359,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,1.0,1.0,0.383333,0.556604,1.0
1,0.512821,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.5,0.666667,0.0,1.0,1.0,0.216667,0.575472,1.0
2,0.717949,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.666667,1.0,0.666667,0.0,1.0,1.0,0.216667,0.698113,1.0
3,0.717949,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,1.0,1.0,0.016667,0.783019,1.0
4,0.435897,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,1.0,1.0,0.033333,0.462264,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,0.820513,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.666667,0.0,1.0,1.0,0.000000,0.452830,1.0
4020,0.666667,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.333333,0.5,0.666667,0.0,1.0,1.0,0.216667,0.641509,1.0
4021,0.974359,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.666667,0.0,1.0,0.0,0.166667,0.641509,1.0
4022,0.717949,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.333333,0.0,0.666667,0.0,1.0,1.0,0.166667,0.669811,1.0


In [ ]:
df_filtered.var()


Age                       0.052819
White                     0.128816
Black                     0.067103
Other                     0.073217
Married                   0.225467
Divorced                  0.106215
Single                    0.129507
Widowed                   0.055003
Separated                 0.011061
T Stage                   0.065115
N Stage                   0.120228
differentiate             0.045260
A Stage                   0.022346
Estrogen Status           0.062396
Progesterone Status       0.143407
Regional Node Examined    0.018224
Survival Months           0.046760
Status                    0.129680
dtype: float64

In [ ]:
#Feature Ranking and Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
#create features and label
X = df_filtered.iloc[:, 0:len(df_filtered.columns)-1]
y = df_filtered.Status

In [ ]:
#sequential feature selection for 10 features
knn = KNeighborsClassifier(n_neighbors=20)
sfs = SequentialFeatureSelector(knn, n_features_to_select=10)
sfs.fit(X, y)
print(sfs.get_support())
X = sfs.transform(X)
X.shape


[ True  True  True  True False False False False  True False  True False
  True False  True  True  True]


(4024, 10)

In [ ]:
#see selected features
support = sfs.get_support()
for i in range(0, len(support)):
  print(f"{i} {df_filtered.columns[i]} {support[i]}")

0 Age True
1 White True
2 Black True
3 Other True
4 Married False
5 Divorced False
6 Single False
7 Widowed False
8 Separated True
9 T Stage False
10 N Stage True
11 differentiate False
12 A Stage True
13 Estrogen Status False
14 Progesterone Status True
15 Regional Node Examined True
16 Survival Months True


# KNN

In [ ]:
from sklearn.base import BaseEstimator
# BaseEstimator doesn't actually do any behavior, we implemented all KNN behavior from scratch below.
# It just allows us to use our self-made KNN model work with sci-kit learns cross-validation so we don't have to write that separately.
class KNN(BaseEstimator):
  def __init__(self,k = 10):
    self.k = k

  def fit(self,X,y):
    self.X_train = X
    self.y_train = y.to_numpy()
    self.is_fitted_ = True
    return self

  def query(self, query):
    distances = (self.X_train - query) ** 2
    distances = np.sum(distances,axis=1)
    indices = np.argsort(distances)[0:self.k]
    total_true = np.sum(self.y_train[indices])
    if total_true > self.k // 2:
      return 1
    else:
      return 0

  def predict(self,Q):
    preds = np.zeros(len(Q))
    for i in range(len(Q)):
      preds[i] = self.query(Q[i])

    return preds

  def score(self,queries,answers):
    # print(queries.shape)
    preds = np.zeros(len(queries))
    for i in range(len(queries)):
      preds[i] = self.query(queries[i])
    return np.count_nonzero(preds == answers) / len(queries)






In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
knn_model = KNN()
KNN_accuracy_scores = cross_val_score(knn_model, X, y, cv=10)
KNN_precision_scores = cross_val_score(knn_model, X, y, cv=10, scoring='precision_macro')
KNN_recall_scores = cross_val_score(knn_model, X, y, cv=10, scoring='recall_macro')
KNN_f1_scores = cross_val_score(knn_model, X, y, cv=10, scoring='f1_macro')
print("Mean accuracy:", KNN_accuracy_scores.mean())
print("Mean precision:", KNN_precision_scores.mean())
print("Mean recall:", KNN_recall_scores.mean())
print("Mean f1:", KNN_f1_scores.mean())

Mean accuracy: 0.8916484574645382
Mean precision: 0.8310562172325107
Mean recall: 0.7023437979784898
Mean f1: 0.7416226732725535


# Naive Bayes

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

#gaussian naive bayes on 10 fold cross validation
NB_accuracy_scores = cross_val_score(gnb, X, y, cv=10)
NB_precision_scores = cross_val_score(gnb, X, y, cv=10, scoring='precision_macro')
NB_recall_scores = cross_val_score(gnb, X, y, cv=10, scoring='recall_macro')
NB_f1_scores = cross_val_score(gnb, X, y, cv=10, scoring='f1_macro')
print("Mean accuracy:", NB_accuracy_scores.mean())
print("Mean precision:", NB_precision_scores.mean())
print("Mean recall:", NB_recall_scores.mean())
print("Mean f1:", NB_f1_scores.mean())

Mean accuracy: 0.852638173894794
Mean precision: 0.7113801736960311
Mean recall: 0.6762860948427253
Mean f1: 0.6894802941962168


Naive bayes assumes features are independent of each other. In this case, this is a valid assumption as in preprocessing we removed highly correlated features. It is computationally efficient compared to many other predictive classification models. Naive bayes has no hyperparameters.

[Source](https://www.upgrad.com/blog/naive-bayes-classifier/://)

# C4.5 Decision Tree

In [ ]:
from sklearn import tree

#decision tree on 10 fold cross validation
clf = tree.DecisionTreeClassifier()
clf_accuracy_scores = cross_val_score(clf, X, y, cv=10)
clf_precision_scores = cross_val_score(clf, X, y, cv=10, scoring='precision_macro')
clf_recall_scores = cross_val_score(clf, X, y, cv=10, scoring='recall_macro')
clf_f1_scores = cross_val_score(clf, X, y, cv=10, scoring='f1_macro')
print("Mean accuracy:", clf_accuracy_scores.mean())
print("Mean precision:", clf_precision_scores.mean())
print("Mean recall:", clf_recall_scores.mean())
print("Mean f1:", clf_f1_scores.mean())

Mean accuracy: 0.8414435267829588
Mean precision: 0.692734569570663
Mean recall: 0.7107483237514034
Mean f1: 0.7013779851089701


The decision tree results are seen above. Max depth is one of its hyperparameters which controls how complicated the tree can become. A higher depth can lead to overfitting. Decision trees in general need little preprocessing to be effective, however small changes in its hyperparameters can lead to drastically different results.

Sources: [Article on Decision Trees](https://dhirajkumarblog.medium.com/top-5-advantages-and-disadvantages-of-decision-tree-algorithm-428ebd199d9a)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#random forest on 10 fold cross validation
RandomForest = RandomForestClassifier()
RandomForest_accuracy_scores = cross_val_score(RandomForest, X, y, cv=10)
RandomForest_precision_scores = cross_val_score(RandomForest, X, y, cv=10, scoring='precision_macro')
RandomForest_recall_scores = cross_val_score(RandomForest, X, y, cv=10, scoring='recall_macro')
RandomForest_f1_scores = cross_val_score(RandomForest, X, y, cv=10, scoring='f1_macro')
print("Mean accuracy:", RandomForest_accuracy_scores.mean())
print("Mean precision:", RandomForest_precision_scores.mean())
print("Mean recall:", RandomForest_recall_scores.mean())
print("Mean f1:", RandomForest_f1_scores.mean())


Mean accuracy: 0.8978611903262841
Mean precision: 0.8428205452226049
Mean recall: 0.7350622354128902
Mean f1: 0.7768819370652804


Random forest results are seen above. It did better than decision trees on all counts, and especially better in precision. Similarly to a decision tree an important hyperparameter is max depth of each tree. The number of trees is a hyperparameter that adjust the max number of trees the model will build; generally a higher number of trees leads to more accurate results but longer training time. Again, similarly to trees random forests can handle non-scaled data and non-linear relationships.

Sources: [Medium Article](https://medium.datadriveninvestor.com/random-forest-pros-and-cons-c1c42fb64f04://)

# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier()

#gradient boosting on 10 fold cross validation
GB_accuracy_scores = cross_val_score(GB, X, y, cv=10)
GB_precision_scores = cross_val_score(GB, X, y, cv=10, scoring='precision_macro')
GB_recall_scores = cross_val_score(GB, X, y, cv=10, scoring='recall_macro')
GB_f1_scores = cross_val_score(GB, X, y, cv=10, scoring='f1_macro')
print("Mean accuracy:", GB_accuracy_scores.mean())
print("Mean precision:", GB_precision_scores.mean())
print("Mean recall:", GB_recall_scores.mean())
print("Mean f1:", GB_f1_scores.mean())

Mean accuracy: 0.9013394565633371
Mean precision: 0.8503391857243061
Mean recall: 0.7335120129971127
Mean f1: 0.7739845158033464


Gradient boosting combines ideas of gradient descent with model boosting algorithms to train a series of models which improve slightly based on the previous iteration. Gradient boosting is hard to interpret and computationally expensive. Since this algorithm hinges on the idea of gradient, its hyperparameter learning rate is important in determing the models next iteration.

# Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#Hyperparameter tuning for Random Forests: learn rate and number of boosting stages (n_estimators)

GB_Tuning = GradientBoostingClassifier()

search_space = dict()
#parameters to tune
search_space['n_estimators'] = [n*10 for n in range(1,50)]
search_space['learning_rate'] = [n*0.01 for n in range(1,50)]

#scoring metric with accuracy, take 50 iterations and 10 fold cross validation
GBrandomizedSearch = RandomizedSearchCV(GB_Tuning, param_distributions=search_space, n_iter=50, scoring='accuracy', cv=10)

GBrandomizedSearch.fit(X, y)

print("Best hyperparameters:", GBrandomizedSearch.best_params_)
print("Best score:", GBrandomizedSearch.best_score_)

Best hyperparameters: {'n_estimators': 310, 'learning_rate': 0.02}
Best score: 0.9048189573225682


In [ ]:
#hyperparameter tuning for Random Forest: max depth and number of trees
RF_Tuning = RandomForestClassifier()
search_space = dict()
#parameters to tune
search_space['n_estimators'] = [n for n in range(1,101)]
search_space['max_depth'] = [n for n in range(1,20)]

#scoring metric with accuracy, take 50 iterations and 10 fold cross validation
RFrandomizedSearch = RandomizedSearchCV(RF_Tuning, param_distributions=search_space, n_iter=50, scoring='accuracy', cv=10)

RFrandomizedSearch.fit(X, y)

print("Best hyperparameters:", RFrandomizedSearch.best_params_)
print("Best score:", RFrandomizedSearch.best_score_)

Best hyperparameters: {'n_estimators': 34, 'max_depth': 5}
Best score: 0.907804031949434
